In [1]:
#coding=utf-8  
from numpy import *  
  
#初始化数据  
#DataList 多条样例数据  
#DataClass 数据对应的分类  
def InitData():  
    DataList=[['data8'],  
              ['data1', 'data2', 'data3', 'data4'],  
              ['data1', 'data2', 'data5', 'data6'],  
              ['data1', 'data2', 'data4', 'data6'],  
              ['data8', 'data9'],  
              ['data7', 'data8'],  
              ['data1', 'data2'],  
              ['data-invalid']]  
    DataClass = [1,1,2,1,2,1,2,0]      
    return DataList,DataClass  
  
#去除重复数据  
#UniqueSet  无重复的数据集  
def UniqueSet(DataSet):  
    UniqueSet = set([])  
    for i in DataSet:  
        UniqueSet = UniqueSet | set(i)#set并  
    return list(UniqueSet)  
      
  
#数据状态压缩  
#InputData 输入数据  
#DataSet   数据集  
#ModelType 模型类别 1：只考虑存不存在  2：考虑出现次数……  
  
#CountArr  数据状态数组  
def GetCountArr(InputData,DataSet,ModelType):  
    CountArr = [0] * len(DataSet)  
    for i in InputData:  
        if i in DataSet:  
            if ModelType == 1:  
                CountArr[DataSet.index(i)] = 1  
            else:  
                CountArr[DataSet.index(i)] += 1  
        else:  
            print(i,' is Invalid data')  
    return array(CountArr)  
  
  
  
              
  
  
  
#朴素贝叶斯分类器      
#CountArrList 数据状态集  
#DataClass    数据类别  
#p1           p1概率自然对数  
#p2           p2概率自然对数  
#p1_ratio     数据集p1占比  
#p2_ratio     数据集p2占比  
#p1Num        数据集影响p1的数量  
#p2Num        数据集影响p2的数量  
def NBM_Classifier(CountArrList,DataClass):  
    ListLen = len(CountArrList)  
    WordsLen = len(CountArrList[0])  
    p1_ratio = (sum(DataClass==1))/float(ListLen)  
    p2_ratio = (sum(DataClass==2))/float(ListLen)  
  
    #某分类下某词项出现频次为0时，其概率也是0，  
    #因此在计算p(w0|ci)p(w1|ci)p(w2|ci)......p(wN|ci)会因为其中某个的概率为0而全部是0。   
    #为了避免这样的情况发生，我们将所有词项出现的频次都初始化为1，某类所有词项数量初始化为2。  
    p1Num = ones(WordsLen)  
    p2Num = ones(WordsLen)  
    p1Denom = 2.0  
    p2Denom = 2.0  
    for i in range(ListLen):  
        if DataClass[i] == 1:  
            p1Num +=CountArrList[i]  
            p1Denom += sum(CountArrList[i])  
        elif DataClass[i] == 2:  
            p2Num +=CountArrList[i]  
            p2Denom += sum(CountArrList[i])  
  
    #由于p(w0|ci)p(w1|ci)p(w2|ci)......p(wN|ci)中每个因子都很小  
    #所有因子相乘，特别是因子数量多的时候，会导致结果溢出，从而得到错误的数据   
    #为了避免溢出问题的发生，使用求自然对数的方法  
    #自然对数和原本的数值同增同减，不会有任何损失，因此不会影响求得的概率结果。  
  
  
    print(p1Num/p1Denom)  
    print(p2Num/p2Denom)  
    p1 = log(p1Num/p1Denom)  
    p2 = log(p2Num/p2Denom)  
    return p1,p2,p1_ratio,p2_ratio,p1Num-1,p2Num-1  
      
  
#获取数据类别  
#CountArr  数据状态数组  
#p1        p1概率自然对数  
#p2        p2概率自然对数  
#p1_ratio  数据集p1占比  
#p2_ratio  数据集p2占比  
#p1Num     数据集影响p1的数量  
#p2Num     数据集影响p2的数量  
def GetDataClass(CountArr, p1, p2, p1_ratio,p2_ratio,p1Num,p2Num):  
  
    #无效数据：该数据没有影响过p1和p2  
    if sum(CountArr*p1Num) == 0 and sum(CountArr*p2Num) == 0:  
        return -1  
    #数学公式：ln(a*b)=ln(a) +ln(b)  
    is_p1 = sum(CountArr * p1) + log(p1_ratio)     
    is_p2 = sum(CountArr * p2) + log(p2_ratio)  
  
    if is_p1 > is_p2:          
        return 1  
    else:   
        return 2   
  
  
  
  
          
def Test():  
    DataList,DataClass = InitData()  
    UniqueDataList = UniqueSet(DataList)  
    ModelType = 1  
    CountArrList = []  
    for InputData in DataList:  
        CountArrList.append(GetCountArr(InputData,UniqueDataList,ModelType))  
    p1, p2, p1_ratio,p2_ratio,p1Num,p2Num= NBM_Classifier(array(CountArrList),array(DataClass))  
  
    TestData =[['data-invalid','data0'],  
    ['data1','data2','data3','data5'],  
    ['data4','data6'],  
    ['data8'],  
    ['data7', 'data8','data0']]  
  
  
    for InputData in TestData:  
        CountArr = GetCountArr(InputData,UniqueDataList,ModelType)  
        print(InputData,' is class ',GetDataClass(CountArr,p1,p2,p1_ratio,p2_ratio,p1Num,p2Num))  
   
Test()  

[0.23076923 0.07692308 0.23076923 0.15384615 0.15384615 0.07692308
 0.23076923 0.15384615 0.07692308 0.23076923]
[0.2 0.1 0.1 0.2 0.1 0.2 0.3 0.1 0.2 0.3]
data0  is Invalid data
['data-invalid', 'data0']  is class  -1
['data1', 'data2', 'data3', 'data5']  is class  2
['data4', 'data6']  is class  1
['data8']  is class  1
data0  is Invalid data
['data7', 'data8', 'data0']  is class  1
